## Opening Suite Generator

Generates unique opening positions based on the neural network weights which
was trained on the lichess.org database.
Every move a new move is sampled from the neural net probability distribtions and applied to the game state.
If the new resulting position has not been added to the suite before, the move ordering is added to the opening suite.
Otherwise a new move is sampled from the resulting position.
No noise is applied to the probability distribution

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%reload_ext autoreload

In [ ]:
import chess
import chess.variant
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0,'../../')
from DeepCrazyhouse.src.runtime.color_logger import enable_color_logging
from DeepCrazyhouse.src.domain.agent.neural_net_api import NeuralNetAPI
from DeepCrazyhouse.src.domain.agent.player.mcts_agent import MCTSAgent
from DeepCrazyhouse.src.domain.agent.player.raw_net_agent import RawNetAgent
from DeepCrazyhouse.src.domain.variants.game_state import GameState

from tqdm import tnrange, tqdm_notebook
import chess.pgn
from time import time
%matplotlib inline
plt.style.use('seaborn-whitegrid')

In [ ]:
batch_size = 8 #1 #8
alpha = 0.2
epsilon = 0.25
nb_openings = int(1e3)

In [ ]:
net = NeuralNetAPI(ctx='cpu', batch_size=batch_size)

In [ ]:
raw_agent = RawNetAgent(net)

In [ ]:
#board = chess.variant.CrazyhouseBoard()
board = chess.variant.AtomicBoard()
state = GameState(board)
board

In [ ]:
def plot_moves_with_prob(moves, probs, only_top_x=None):
    
    # revert the ordering afterwards
    idx_order = np.argsort(probs)[::-1]
    
    if only_top_x is not None and only_top_x < len(idx_order):
        idx_order = idx_order[:only_top_x]
    
    #moves_ordered = moves[range(len(moves))] #idx_order[::-1]]
    probs_ordered = [] #probs[idx_order]
    
    moves_ordered = []
    for idx in idx_order:
        probs_ordered.append(probs[idx])
        moves_ordered.append(moves[idx])
        
    plt.barh(range(len(probs_ordered)), probs_ordered)
    plt.yticks(range(len(moves_ordered)), moves_ordered)


In [ ]:
t_s = time()
pred_value, legal_moves, p_vec_small, cp, depth, nodes, time_elapsed_s, nps, pv = raw_agent.evaluate_board_state(state)
print('Elapsed time: %.4fs' % (time()-t_s))

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
dirichlet_noise = np.random.dirichlet([alpha] * len(legal_moves))
#dirichlet_noise = np.ones(len(legal_moves)) * 1/len(legal_moves)
p_vec_small = (1 - epsilon) * p_vec_small + epsilon * dirichlet_noise

In [ ]:
plot_moves_with_prob(legal_moves, p_vec_small, only_top_x=10)

In [ ]:
opening_states = {}

In [ ]:
plys_state = np.zeros(nb_openings)
for i in tnrange(nb_openings, desc='generated openings'):
    # create a new entry in the dictionary
    state.new_game()
    game = chess.pgn.Game()
    game.headers["Event"] = "Lichess Opening Explorer"# (0.25 Dirichlet Noise)"
    game.headers["Variant"] = "atomic" #"crazyhouse"
    game.headers["Date"] = "2020.10.26" #"2018.12.31"
    plys = 0

    node = None
    while True:
        plys += 1
        pred_value, legal_moves, p_vec_small, cp, depth, nodes, time_elapsed_s, nps, pv = raw_agent.evaluate_board_state(state)
        #dirichlet_noise = np.random.dirichlet([alpha] * len(legal_moves))
        #p_vec_small = (1 - epsilon) * p_vec_small + epsilon * dirichlet_noise

        # make sure the probabilities will sum up to 1
        #p_vec_small /= p_vec_small.sum()
        sel_move = np.random.choice(legal_moves, p=p_vec_small)
        
        if node is None:
            # create a new node
            node = game.add_variation(sel_move)
        else:
            # add the move to the existing node
            node = node.add_variation(sel_move)

        #print(sel_move.uci(), end=" ")
        state.apply_move(sel_move)
        transpos_key = state.get_transposition_key()
        if transpos_key not in opening_states:
            opening_states[transpos_key] = True
            plys_state[i] = plys
            break
    #print()
    
    with open('lichess_atomic_startpos.pgn', 'a') as f:
        f.write(str(game) +'\n\n')


In [ ]:
print('Statistics: Half Moves %.2f +/- %.2f' % (plys_state.mean(), plys_state.std()))